In [4]:
# Cell 1: Imports & Helpers
import os
import json
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import shutil

# order four points into TL,TR,BR,BL
def order_points(pts):
    s    = pts.sum(axis=1)
    diff = np.diff(pts, axis=1).ravel()
    return np.array([
        pts[np.argmin(s)],      # TL
        pts[np.argmin(diff)],   # TR
        pts[np.argmax(s)],      # BR
        pts[np.argmax(diff)]    # BL
    ], dtype="float32")

# warp a bounding‐box under perspective M
def warp_bbox(bbox, M):
    # bbox = [x1,y1,x2,y2]
    pts = np.array([[[bbox[0],bbox[1]],
                     [bbox[0],bbox[3]],
                     [bbox[2],bbox[1]],
                     [bbox[2],bbox[3]]]], dtype="float32")
    warped = cv2.perspectiveTransform(pts, M)[0]
    xs, ys = warped[:,0], warped[:,1]
    return xs.min(), ys.min(), xs.max(), ys.max()

In [5]:
# Cell 2: Paths & params
INPUT_RGB  = "/kaggle/input/chessrender360/ChessRender360/rgb"
INPUT_ANN  = "/kaggle/input/chessrender360/ChessRender360/annotations"
OUTPUT     = "/kaggle/working/chess360_yolo"
SQUARE_SZ  = 800   # size of warped board
os.makedirs(OUTPUT, exist_ok=True)
# 12 classes by piece_id
CLASS_NAMES = [
  'white_queen','white_king','white_bishop','white_knight',
  'white_rook','white_pawn',
  'black_queen','black_king','black_bishop','black_knight',
  'black_rook','black_pawn'
]

In [6]:
# Cell 3: Gather IDs and train/val split
all_ids = []
for fn in os.listdir(INPUT_ANN):
    if fn.startswith("annotation_") and fn.endswith(".json"):
        idx = fn[len("annotation_"):-len(".json")]
        rgb_path = os.path.join(INPUT_RGB, f"rgb_{idx}.jpeg")
        if os.path.exists(rgb_path):
            all_ids.append(idx)
train_ids, val_ids = train_test_split(all_ids, test_size=0.2, random_state=42)

In [7]:
# Cell 4: Make output folders
for split in ("train","val"):
    for sub in ("images","labels"):
        os.makedirs(os.path.join(OUTPUT,split,sub), exist_ok=True)

In [19]:
def convert_with_padding(split_ids, split):
    pad_cells = 1                
    pad_px    = SQUARE_SZ // 8 * pad_cells
    out_sz    = SQUARE_SZ + 2*pad_px

    for idx in split_ids:
        js  = json.load(open(f"{INPUT_ANN}/annotation_{idx}.json"))
        img = cv2.imread(f"{INPUT_RGB}/rgb_{idx}.jpeg")
        h,w = img.shape[:2]

        # 1) get & order source corners
        bc  = js["board_corners"]
        src = np.array([
            bc["white_left"],
            bc["white_right"],
            bc["black_right"],
            bc["black_left"]
        ], dtype="float32")
        src = order_points(src)

        # 2) build destination quad with positive padding
        dst = np.array([
            [    pad_px,     pad_px],
            [SQUARE_SZ+pad_px,     pad_px],
            [SQUARE_SZ+pad_px, SQUARE_SZ+pad_px],
            [    pad_px,     SQUARE_SZ+pad_px]
        ], dtype="float32")

        # 3) warp to include one-square border on *every* side
        M      = cv2.getPerspectiveTransform(src, dst)
        warped = cv2.warpPerspective(img, M, (out_sz, out_sz))
        cv2.imwrite(f"{OUTPUT}/{split}/images/{idx}.jpg", warped)

        # 4) warp each piece bbox and write labels (same as before)
        lines = []
        for p in js["pieces"]:
            x1,y1,x2,y2 = p["bbox"]
            wx1,wy1,wx2,wy2 = warp_bbox([x1,y1,x2,y2], M)
            wx1,wy1 = max(0,wx1), max(0,wy1)
            wx2,wy2 = min(out_sz,wx2), min(out_sz,wy2)
            cx = ((wx1+wx2)/2) / out_sz
            cy = ((wy1+wy2)/2) / out_sz
            bw = (wx2-wx1)    / out_sz
            bh = (wy2-wy1)    / out_sz
            cls= p["piece_id"]
            lines.append(f"{cls} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")

        with open(f"{OUTPUT}/{split}/labels/{idx}.txt","w") as f:
            f.write("\n".join(lines))
convert_with_padding(train_ids, "train")
convert_with_padding(val_ids,   "val")

In [20]:
# Cell 6: data.yaml
data_yaml = f"""
train: {OUTPUT}/train/images
val:   {OUTPUT}/val/images

nc: {len(CLASS_NAMES)}
names: {CLASS_NAMES}
""".strip()

with open(os.path.join(OUTPUT,"data.yaml"),"w") as f:
    f.write(data_yaml)

print("✅ data.yaml:\n", data_yaml)

✅ data.yaml:
 train: /kaggle/working/chess360_yolo/train/images
val:   /kaggle/working/chess360_yolo/val/images

nc: 12
names: ['white_queen', 'white_king', 'white_bishop', 'white_knight', 'white_rook', 'white_pawn', 'black_queen', 'black_king', 'black_bishop', 'black_knight', 'black_rook', 'black_pawn']


In [21]:
# Cell 7: Zip output for download
shutil.make_archive("/kaggle/working/chess360_yolo", "zip", OUTPUT)
print("Zipped to /kaggle/working/chess360_yolo.zip")

Zipped to /kaggle/working/chess360_yolo.zip
